In [ ]:
using LinearAlgebra

# Function to compute the leading eigenvalue and eigenvector using the power method
function power_method_eigen(A::AbstractMatrix{T}; num_iterations::Int=1000, tolerance::Float64=1e-6, round_digits::Int=4) where T<:Real
    # Ensure that A is square
    n, m = size(A)
    if n != m
        error("Matrix A must be square.")
    end
    # Initialize and normalize a random vector
    x = rand(n) 
    x /= norm(x)

    eigenval_old = 0.0
    eigenval = 0.0

    for i in 1:num_iterations 
        # Compute new vector 
        x_new = A * x
        # Compute the Rayleigh quotient as an estimate of the eigenvalue
        eigenval = dot(x_new, x) 
        # Normalize new vector 
        x_new /= norm(x_new) 
        # If old eigenvalue approx = new eigenvalue, terminate
        if abs(eigenval - eigenval_old) < tolerance
            break
        end 
        # set values for next iteration
        x = x_new
        eigenval_old = eigenval
    end
    # round eigenvalue and eigenvector
    eigenval = round(eigenval, digits=round_digits)
    x = round.(x, digits=round_digits)

    return eigenval, x
end

# Testing 
A = [8 -6 2;
    -6 7 -4;
    2 -4 3];
eigenval, v = power_method_eigen(A)
println("Leading eigenvalue: ", eigenval)
println("Leading eigenvector: ", v)


A = [5 -5 5;
    -5 0 0;
    5 0 0];
eigenval, v = power_method_eigen(A)
println("Leading eigenvalue: ", eigenval)
println("Leading eigenvector: ", v)


Leading eigenvalue: 15.0
Leading eigenvector: [0.6667, -0.6667, 0.3333]
Leading eigenvalue: 10.0
Leading eigenvector: [0.8166, -0.4082, 0.4082]
